In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, LongType
import codecs

In [3]:
def loadMovieNames():
    movieNames = {}
    with codecs.open("/home/turan/SparkCourse/section_4/ml-100k/u.item", "r", encoding='ISO-8859-1', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

In [4]:
spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

25/11/18 11:46:26 WARN Utils: Your hostname, turan-H110 resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface enp2s0)
25/11/18 11:46:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/18 11:46:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/18 11:46:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
nameDict = spark.sparkContext.broadcast(loadMovieNames())

In [6]:
schema = StructType([ \
    StructField("userID", IntegerType(), True),\
    StructField("movieID", IntegerType(), True),\
    StructField("rating", IntegerType(), True),\
    StructField("timestamp", LongType(), True)])

In [7]:
moviesDF = spark.read.option("sep", "\t").schema(schema).csv("file:///home/turan/SparkCourse/section_4/ml-100k/u.data")

In [8]:
movieCounts = moviesDF.groupBy("movieID").count()

In [9]:
def lookupName(movieID):
    return nameDict.value[movieID]

In [10]:
lookupNameUDF = func.udf(lookupName)

In [11]:
moviesWithNames = movieCounts.withColumn("movieTitle", lookupNameUDF(func.col("movieID")))

In [12]:
sortedMoviesWithNames = moviesWithNames.orderBy(func.desc("count"))

In [13]:
sortedMoviesWithNames.show(10, False)

+-------+-----+-----------------------------+
|movieID|count|movieTitle                   |
+-------+-----+-----------------------------+
|50     |583  |Star Wars (1977)             |
|258    |509  |Contact (1997)               |
|100    |508  |Fargo (1996)                 |
|181    |507  |Return of the Jedi (1983)    |
|294    |485  |Liar Liar (1997)             |
|286    |481  |English Patient, The (1996)  |
|288    |478  |Scream (1996)                |
|1      |452  |Toy Story (1995)             |
|300    |431  |Air Force One (1997)         |
|121    |429  |Independence Day (ID4) (1996)|
+-------+-----+-----------------------------+
only showing top 10 rows



In [14]:
spark.stop()